# Charting Example

## Basic Setup

### Load Packages

In [55]:
import os
import pandas as pd
import plotly.express as px
from datetime import datetime
from dateutil.relativedelta import relativedelta
from math import ceil

pd.options.mode.copy_on_write = True

#### Use ```os``` to manage project files and load dataset

In [3]:
os.getcwd()
file = "Trip and visit WY24_25 Database.xlsx"
df = pd.read_excel(os.path.join(os.getcwd(), file))
df.dropna(inplace=True)

df.head(3)

/Users/Chin2/Documents/projects/work-trips/work-trips/venv/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,S/N,Country,Service,Trip or Visit,Departure Date,Return Date,Status,Level,Del Head
0,1,Thailand,ABC,Trip (Outgoing),2024-05-01,2024-05-02,Cancelled,Director,ADS
1,2,Brunei,DEF,Trip (Outgoing),2024-05-29,2024-06-01,Completed,BB,FDW
2,3,France,GHI,Trip (Outgoing),2024-09-19,2024-09-20,Planned,HOD,DSG


#### Columns Renaming (for easier usage with Pandas)

In [4]:
### Rename Columns to be lowercased + no spacing --> Easier to select columns

rename_columns = {
    'S/N' : 'sn',
    'Country': 'country',
    'Service': 'service',
    'Trip or Visit': 'trip_or_visit',
    'Departure Date': 'departure_date',
    'Return Date': 'return_date',
    'Status': 'status',
    'Level': 'level',
    'Del Head': 'del_head',
}

df.rename(columns=rename_columns, inplace=True)

## Examples

#### Pandas DataFrame ```loc``` example

In [55]:
### Using Pandas Loc to perform filtering or select columns
selected_country = "Thailand"
selected_service = "ABC"
redundant_columns = ["sn", "del_head"]

df.loc[
    ## Filter Row
    (df.country == selected_country) & (df.service == selected_service),
    
    ## Filter COlumn
    [i for i in df.columns.to_list() if i not in redundant_columns]
]

,country,service,trip_or_visit,departure_date,return_date,status,level,Month
0,Thailand,ABC,Trip (Outgoing),2024-05-01,2024-05-02,Cancelled,Director,May


#### Function to do filtering on multiple columns on a Dataset

In [5]:
def filter_dataset(df, **kwargs):
    query_items = {}
    for k, v in kwargs.items():
        query_items[k] = v
    
    query = " & ".join([
            f"({k} == '{v}')" 
            if (type(v) == str)
            else
            f"({k} in {v})"
            for k,v in query_items.items()
        ])
    
    print(f"Query:     {query}")
    try:
        return df.query(query)
    except:
        print("KeyError - Please make sure the columns exist in the dataframe!")

In [58]:
filter_dataset(df, country="USA", service=["ABC"])

Query:     (country == 'USA') & (service in ['ABC'])


,sn,country,service,trip_or_visit,departure_date,return_date,status,level,del_head,Month


#### Function to do Value Count (by Year/Month) on a Date Column

In [35]:
def prepare_value_count_time_series_plot(
  df,
  date_column,
  start_date,
  end_date,
  type = "monthly"
):
    # filter date range
    df_subset =  df[(df[date_column] >= start_date) & (df[date_column] <= end_date)]
    
    # add date info
    df_subset["year"] = df_subset.loc[:,date_column].dt.year
    df_subset["month"] = df_subset.loc[:,date_column].dt.month
    
    if len(df_subset) == 0:
      df_subset["date"] = pd.NA
      df_subset["label"] = pd.NA
      return df_subset
    
    if type == "yearly":
      df_output = df_subset["year"].value_counts().reset_index(drop=False)
      df_output.sort_values("year", inplace=True)
      return df_output
    else:
      df_subset["date"] = df_subset.apply(lambda x: datetime(x["year"], x["month"], 1), axis=1)
      df_output = df_subset["date"].value_counts().reset_index(drop=False)
      df_output["label"] = df_output["date"].apply(lambda x: x.strftime("%b %Y"))
      df_output.sort_values("date", inplace=True)
      return df_output

In [28]:
prepare_value_count_time_series_plot(df, date_column="departure_date", start_date="2024-01-01", end_date="2024-09-01")

,date,count,label
2,2024-02-01,1,Feb 2024
1,2024-04-01,2,Apr 2024
0,2024-05-01,3,May 2024


In [36]:
prepare_value_count_time_series_plot(df, date_column="return_date", start_date="2023-01-01", end_date="2025-09-01", type="yearly")

,year,count
0,2024,11
1,2025,1


## Create Trip vs Visit Time Series Chart

In [39]:
# Separate 2 types into 2 different DataFrames
df_trip = df.query("trip_or_visit == 'Trip (Outgoing)'")
df_visit = df.query("trip_or_visit == 'Visit (Incoming)'")

# Create Value Count Dataset of the 2 types
df_visit_count = prepare_value_count_time_series_plot(df_visit, date_column="departure_date", start_date="2021-03-01", end_date="2025-5-21")
df_trip_count = prepare_value_count_time_series_plot(df_trip, date_column="departure_date", start_date="2021-03-01", end_date="2025-5-21")

# Add variable to identity type of event
df_visit_count["type"] = "visit"
df_trip_count["type"] = "trip"

# Combine 2 DataFrames
df_combined_count = pd.concat([
    df_trip_count,
    df_visit_count
])

df_combined_count.reset_index(drop=True, inplace=True)
df_combined_count.sort_values(by="date", inplace=True)
df_combined_count

,date,count,label,type
5,2024-02-01,1,Feb 2024,visit
0,2024-04-01,2,Apr 2024,trip
1,2024-05-01,3,May 2024,trip
2,2024-09-01,2,Sep 2024,trip
3,2024-10-01,1,Oct 2024,trip
4,2024-11-01,1,Nov 2024,trip
6,2024-11-01,1,Nov 2024,visit


In [64]:
df_combined_count.date.max()

Timestamp('2024-11-01 00:00:00')

In [76]:
# Additional Chart Related Data


# Create List of Axis Labels
min_count = df_combined_count["count"].min()
max_count = df_combined_count["count"].max()

# Create List of Axis Labels
earliest_date = df_combined_count.date.min()
latest_date = df_combined_count.date.max()
delta = latest_date - earliest_date
months_delta = ceil((delta.days) / 30)

list_of_xlabel_values = []
list_of_xlabel_text = []

for i in range(0, months_delta):
    curr_date = earliest_date + relativedelta(months=i)
    list_of_xlabel_values.append(curr_date)
    list_of_xlabel_text.append(curr_date.strftime("%b %Y"))
    
list_of_xlabel_text

['Feb 2024',
 'Mar 2024',
 'Apr 2024',
 'May 2024',
 'Jun 2024',
 'Jul 2024',
 'Aug 2024',
 'Sep 2024',
 'Oct 2024',
 'Nov 2024']

In [77]:
# Plot

fig = px.line(
    df_combined_count,
    x = 'date',
    y = 'count',
    color= "type",
    title = 'Monthly Breakdown',
    markers=True,
    labels= df_combined_count["label"],
)

fig.update_layout(
                  xaxis = dict(
                    tickmode='array',
                    tickvals = list_of_xlabel_values,
                    ticktext = list_of_xlabel_text
                    ),
                  yaxis_range=[min(0, min_count - 1), max_count + 1]
                )

fig.show()

## Other Notes

- Can explore Flask/SQLAlchemy + SQLiteDB for Backend Application Development
- Can read Database Normalization to learn Database Modelling